# Tool Input Schema

By default, tools infer the argument schema by inspecting the function signature. For more strict requirements, custom input schema can be specified, along with custom validation logic.

In [1]:
from typing import Any, Dict

from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools.requests.tool import RequestsGetTool, TextRequestsWrapper
from pydantic import BaseModel, Field, root_validator


In [2]:
llm = OpenAI(temperature=0)

In [2]:
!pip install tldextract > /dev/null

In [3]:
import tldextract

_APPROVED_DOMAINS = {
    "langchain",
    "wikipedia",
}

class ToolInputSchema(BaseModel):

    url: str = Field(...)
    
    @root_validator
    def validate_query(cls, values: Dict[str, Any]) -> Dict:
        url = values["url"]
        domain = tldextract.extract(url).domain
        if domain not in _APPROVED_DOMAINS:
            raise ValueError(f"Domain {domain} is not on the approved list:"
                             f" {sorted(_APPROVED_DOMAINS)}")
        return values
    
tool = RequestsGetTool(args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper())

In [5]:
agent = initialize_agent([tool], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

In [6]:
# This will succeed, since there aren't any arguments that will be triggered during validation
answer = agent.run("What's the main title on langchain.com?")
print(answer)

The main title of langchain.com is "LANG CHAIN 🦜️🔗 Official Home Page"


In [6]:
agent.run("What's the main title on google.com?")

ValidationError: 1 validation error for ToolInputSchema
__root__
  Domain google is not on the approved list: ['langchain', 'wikipedia'] (type=value_error)

In [4]:
from langchain.callbacks.human import HumanApprovalCallbackHandler

In [5]:
tool = RequestsGetTool(args_schema=ToolInputSchema, requests_wrapper=TextRequestsWrapper(), callbacks=[HumanApprovalCallbackHandler()])

In [6]:
tool.run(tool_input={"url": "https://langchain.com"})

Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

{'url': 'https://langchain.com'}
yes


'\n<p align="center">\n\t<b><font size="6">L</font><font size="4">ANG </font><font size="6">C</font><font size="4">HAIN </font><font size="2">🦜️🔗</font><br>Official Home Page</b><font size="1">&nbsp;</font></p>\n\n<hr>\n<center>\n<table border="0" cellspacing="0" width="90%">\n  <tbody>\n  <tr>\n    <td height="55" valign="top" width="50%">\n      <ul>\n        <li><a href="integrations.html">Integrations</a> \n    </li></ul></td>\n   <td height="45" valign="top" width="50%">\n      <ul>\n        <li><a href="features.html">Features</a> \n        </li></ul></td></tr>\n    <tr>\n    <td height="55" valign="top" width="50%">\n      <ul>\n        <li><a href="https://blog.langchain.dev/">Blog</a> \n    </li></ul></td>\n   <td height="45" valign="top" width="50%">\n      <ul>\n        <li><a href="https://docs.langchain.com/docs/">Conceptual Guide</a> \n        </li></ul></td></tr>\n\n  <tr>\n    <td height="45" valign="top" width="50%">\n      <ul>\n        <li><a href="https://github.com

In [7]:
tool.run(tool_input={"url": "https://langchain.com"})

Do you approve of the following input? Anything except 'Y'/'Yes' (case-insensitive) will be treated as a no.

{'url': 'https://langchain.com'}
no


InputRejectedException: Inputs {'url': 'https://langchain.com'} to tool {'name': 'requests_get', 'description': 'A portal to the internet. Use this when you need to get specific content from a website. Input should be a  url (i.e. https://www.google.com). The output will be the text response of the GET request.'} were rejected.